In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Customer_DF=pd.read_csv('Never_pay_customer_list.csv')

In [ ]:
Customer_DF.info()

In [ ]:
Customer_DF.head()

In [ ]:
Customer_DF.rename(columns={'CUSTOMER_ID':'Telus_customer_ID'},inplace=True)

In [ ]:
Customer_DF.head()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.Never_pay_customers'

bq_table_instance= bq_client.load_table_from_dataframe(Customer_DF, Table_BQ,job_config=config)

In [ ]:
Query='''



with base_data as (
select BAN, cast(Telus_customer_ID as STRING) as Telus_customer_ID  from `divgpras-pr-579355.SHS.Never_pay_customers`
)


, ADC_data_Sep_2022 as 

(

SELECT distinct
    dealer_customer_id as Telus_ID,
    avg(Login_consistency) as Login_consistency_sep ,
    avg(number_of_login_days) as Login_days_sep,
    avg(Arming_Consistency) as Arming_consistency_sep,
    1 as Sept_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-09-01'
  group by dealer_customer_id
    
)

, ADC_data_oct_2022 as 

(

SELECT distinct
    dealer_customer_id as Telus_ID,
    avg(Login_consistency) as Login_consistency_Oct ,
    avg(number_of_login_days) as Login_days_Oct,
    avg(Arming_Consistency) as Arming_consistency_Oct,
    1 as Oct_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
  group by dealer_customer_id
    
)


, ADC_data_Nov_2022 as 

(

SELECT distinct
    dealer_customer_id as Telus_ID,
    avg(Login_consistency) as Login_consistency_Nov ,
    avg(number_of_login_days) as Login_days_Nov,
    avg(Arming_Consistency) as Arming_consistency_Nov,
    1 as Nov_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-11-01'
  group by dealer_customer_id
    
)



, ADC_data_Dec_2022 as 

(

SELECT distinct
    dealer_customer_id as Telus_ID,
    avg(Login_consistency) as Login_consistency_Dec ,
    avg(number_of_login_days) as Login_days_Dec,
    avg(Arming_Consistency) as Arming_consistency_Dec,
    1 as Dec_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-12-01'
  group by dealer_customer_id
    
)


, ADC_data_Jan_2023 as 

(

SELECT distinct
    dealer_customer_id as Telus_ID,
    avg(Login_consistency) as Login_consistency_Jan ,
    avg(number_of_login_days) as Login_days_Jan,
    avg(Arming_Consistency) as Arming_consistency_Jan,
    1 as Jan_data_flag

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2023-01-01'
  group by dealer_customer_id
    
)


select a.BAN,a.Telus_customer_ID
,b.Login_consistency_sep,b.Login_days_sep,b.Arming_consistency_sep,b.Sept_data_flag
,c.Login_consistency_Oct,c.Login_days_Oct,c.Arming_consistency_Oct,c.Oct_data_flag
,d.Login_consistency_Nov,d.Login_days_Nov,d.Arming_consistency_Nov,d.Nov_data_flag
,e.Login_consistency_Dec,e.Login_days_Dec,e.Arming_consistency_Dec,e.Dec_data_flag
,f.Login_consistency_Jan,f.Login_days_Jan,f.Arming_consistency_Jan,f.Jan_data_flag
from base_data a
left join ADC_data_Sep_2022 b
on a.Telus_customer_ID=b.Telus_ID
left join ADC_data_oct_2022 c
on a.Telus_customer_ID=c.Telus_ID
left join ADC_data_Nov_2022 d
on a.Telus_customer_ID=d.Telus_ID
left join ADC_data_Dec_2022 e
on a.Telus_customer_ID=e.Telus_ID
left join ADC_data_Jan_2023 f
on a.Telus_customer_ID=f.Telus_ID
order by BAN



'''

In [ ]:
DF=extract_bq_data(bq_client, sql=Query)

In [ ]:
DF.info(verbose=True)

In [ ]:
DF.head(10)

In [ ]:

DF['Sep_Usage_Flag']=DF['Login_consistency_sep'].apply(lambda x: 1 if x>0 else 0)
DF['Oct_Usage_Flag']=DF['Login_consistency_Oct'].apply(lambda x: 1 if x>0 else 0)
DF['Nov_Usage_Flag']=DF['Login_consistency_Nov'].apply(lambda x: 1 if x>0 else 0)
DF['Dec_Usage_Flag']=DF['Login_consistency_Dec'].apply(lambda x: 1 if x>0 else 0)
DF['Jan_Usage_Flag']=DF['Login_consistency_Jan'].apply(lambda x: 1 if x>0 else 0)



In [ ]:
DF['Sep_Usage_Flag'].value_counts(normalize=True)*100

In [ ]:
DF['Oct_Usage_Flag'].value_counts(normalize=True)*100

In [ ]:
DF['Jan_Usage_Flag'].value_counts(normalize=True)*100

In [ ]:
DF.head(10)

In [ ]:
DF.to_csv('Never_pay_customer_list_Usage_updated.csv',index=False)